Analisando a página da Drogaria Minas Mais, percebo que ela é uma página dinâmica (tem o botão de "Carregar mais produtos")
Outro fator que também chamou atenção: ao desligar o Javascript da página, ela não exibiu nenhum produto (sequer conseguiu carregar), exibindo uma dependenciaforte do JS
Assim, vou tentar uma abordagem para um HTML dinâmico 



No site há 6118 produtos, sendo que na primeira página há 15 produtos, na segunda há outros 15 produtos e assim por diante 
Temos um total de 408 páginas
Entretanto na página 51 já não encontramos mais produtos (Acredito que o acesso esteja limitado pelo parâmetro Page, como previa o arquivo de treinamento interno do Notion) 


Vou tentar fazer o processo de filtrar por marcas
A marca com mais produtos é a HYPERA com 371 produtos
O layout do site apresenta 15 produtos por página. Como podemos acessar 50 páginas, temos acesso a 750 (50x15) produtos no máximo
Assim filtrar por marca realmente será uma estratégia válida

<label class="vtex-checkbox__label w-100 c-on-base pointer" for="brand-hypera">HYPERA</label>
<label class="vtex-checkbox__label w-100 c-on-base pointer" for="brand-ache-rx">ACHE RX</label>

Cliquei em inspect em cima das marcas e achei isso no código fonte.
Dar um jeito de pegar todas as marcas com essa estrutura

In [1]:
from bs4 import BeautifulSoup
import requests
import json
import re
import pandas as pd
import math
import multiprocessing
import time


In [2]:
def pegar_html(link):  
    req=requests.get(link)
    html=BeautifulSoup(req.text,"html.parser")
    return html


def achar_marcas(html):
    
    marcas=html.find_all("label", attrs={"class":"vtex-checkbox__label w-100 c-on-base pointer"})
    strmarcas=[]
    POP=[]
    REMOVER=[]

    #transformar ps códigos <label ...  em strings
    for elem in marcas:
        strmarcas.append(str(elem))
    

    #achar os índices das labels(em string) que possuem for="category-2...", que não representam marcas
    for i in range(len(strmarcas)):
        if "category-2" in strmarcas[i]:
            POP.append(i)

    #Achar a correlação entre as labels(em string) e as labels normais. Saber qual label normal deve ser removida
    for indice in POP:
        REMOVER.append(marcas[indice])

    for indesejado in REMOVER:
        marcas.remove(indesejado)

    #pegando somente o nome das marcas (retirando toda a estrutura do label)
    TEXTO=[]
    for label in marcas:
        TEXTO.append(label.string)

    #analisando o sorted(TEXTO), percebi que diversos elementos da lista eram duplicados

    #Tirando os nomes duplicados
    TEXTO_S_DUPL=[]
    for elem in TEXTO:
        if elem not in TEXTO_S_DUPL:
            TEXTO_S_DUPL.append(elem) 


    #formatando o nome da marca
    MARCA_PARA_LINK=[]
    for elem in TEXTO_S_DUPL:
        elem=elem.replace(" ","-")
        elem=elem.replace("/","-")
        elem=elem.lower()
        MARCA_PARA_LINK.append(elem)

    #print(strmarcas)
    #print(POP)
    #print(REMOVER)
    return MARCA_PARA_LINK


def achar_marcas_sem_link(html):
    
    marcas=html.find_all("label", attrs={"class":"vtex-checkbox__label w-100 c-on-base pointer"})
    strmarcas=[]
    POP=[]
    REMOVER=[]

    #transformar ps códigos <label ...  em strings
    for elem in marcas:
        strmarcas.append(str(elem))
    

    #achar os índices das labels(em string) que possuem for="category-2...", que não representam marcas
    for i in range(len(strmarcas)):
        if "category-2" in strmarcas[i]:
            POP.append(i)

    #Achar a correlação entre as labels(em string) e as labels normais. Saber qual label normal deve ser removida
    for indice in POP:
        REMOVER.append(marcas[indice])

    for indesejado in REMOVER:
        marcas.remove(indesejado)

    #pegando somente o nome das marcas (retirando toda a estrutura do label)
    TEXTO=[]
    for label in marcas:
        TEXTO.append(label.string)

    #analisando o sorted(TEXTO), percebi que diversos elementos da lista eram duplicados

    #Tirando os nomes duplicados
    TEXTO_S_DUPL=[]
    for elem in TEXTO:
        if elem not in TEXTO_S_DUPL:
            TEXTO_S_DUPL.append(elem)

    return TEXTO_S_DUPL



def link (MARCA_LINK):
    LINK=[]
    for i in MARCA_LINK:
        link=f"https://www.drogariasminasmais.com.br/medicamentos/{i}?initialMap=c&initialQuery=medicamentos&map=category-1,brand"
        link+="&page="
        LINK.append(link)
    return(LINK)


def qtd_produtos(json_pag,marca):
    nome='$ROOT_QUERY.facets({"behavior":"Static","categoryTreeBehavior":"default","hideUnavailableItems":true,"initialAttributes":"c","query":"medicamentos/hypera","selectedFacets":[{"key":"category-1","value":"medicamentos"},{"key":"brand","value":"hypera"}]})@context({"provider":"vtex.search-graphql"})@runtimeMeta({"hash":"d0cbc665c0364a2c77575f998051aa1a9f8a04a60174d78356b18b0406fe48b6"}).facets.3.values({}).'
    for key,values in json_pag.items():
        if key.startswith(nome):
            

            #alterar a formatação da marca
            marca=marca.upper()
            for sub_key,sub_value in values.items():
                if sub_key=='quantity':
                    quantidade=sub_value
                if sub_key=='name' and sub_value==marca:
                    return quantidade
                
def get_json(url):
        requisicao = requests.get(url)
        soup = BeautifulSoup(requisicao.text, 'html.parser')

        string = soup.find_all('script')[-34].text #Converter para arquivo de texto (retira as tags)

        json_file = string.split("__STATE__ = ", 1) #1 é o indice (lista é criada com o split)
        
        return json.loads(json_file[1])



def get_ean(json_pag):
    EAN=[]
    for key,value in json_pag.items():
        if key.startswith('Product:sp-'):
            for sub_key,sub_value in value.items():
                if sub_key == "ean":
                    EAN.append(sub_value)

    return(EAN)


def get_name(json_pag):
    NOMES=[]
    for key,value in json_pag.items():
        if key.startswith('Product:sp-'):
            for sub_key,sub_value in value.items():
                if sub_key=='productName':
                    sub_value=sub_value.split(" -")[0]
                    NOMES.append(sub_value)
    return NOMES


def get_preço_c_desconto(json_pag):
    VENDA=[]
    for key,value in json_pag.items():
        if '$Product:sp-' and '.priceRange.sellingPrice' in key:
            for sub_key,sub_value in value.items():
                if sub_key == 'highPrice':
                    VENDA.append(sub_value)
    return VENDA


def get_preço_s_desconto(json_pag):
    CHEIO=[]
    for key,value in json_pag.items():
        if '$Product:sp-' and '.priceRange.listPrice' in key:
            for sub_key,sub_value in value.items():
                if sub_key == 'highPrice':
                    CHEIO.append(sub_value)
    return CHEIO


def desconto(A,B):
    DESCONTO=[]
    for i in range(len(A)):
        x= B[i] - A[i]
        if x<0:
            x=(-1)*x
            x=round(x,2)
            DESCONTO.append(x)
        else:
            x=round(x,2)
            DESCONTO.append(x)
    return DESCONTO

Analisando a página da Hypera, pelo ThunderClient, posso perceber que todo o dicionário de interesse está na variável __STATE__ 
Mas ele só devolveu informações da primeira página. 

Acredito que o jeito mais facil será fazer requests das páginas dos jsons (v1 ao inspecionar a página) e achar como os links evoluem. A ideia é fazer requests em cima desses links e extrair as informações

In [3]:
colunas=['EAN','Marca','Nome','Preço com desconto','Preço sem desconto','Desconto']
df=pd.DataFrame(columns=colunas)


url='https://www.drogariasminasmais.com.br/medicamentos'
html=pegar_html(url)
marcas=achar_marcas(html)
LINKS=link(marcas)
MARCAS=achar_marcas_sem_link(html)


for j in range(len(LINKS)):
    json_pag=get_json(LINKS[j] + "1")
    json_generico=get_json(LINKS[0] + "1")
    pag=qtd_produtos(json_generico,MARCAS[j])
    print(pag,MARCAS[j])
    pag=pag/15
    pag=math.ceil(pag) + 1
    
    for i in range(1,pag):

        colunas=['EAN','Marca','Nome','Preço com desconto','Preço sem desconto','Desconto']
        df1=pd.DataFrame(columns=colunas)


        json_pag=get_json(LINKS[j] + str(i))
        ean=get_ean(json_pag)
        nomes_produtos=get_name(json_pag)
        p_des=get_preço_c_desconto(json_pag)
        p_s_des=get_preço_s_desconto(json_pag)
        DESCONTO=desconto(p_s_des,p_des)

        df1['EAN']=ean
        df1['Marca']=marcas[j]
        df1['Nome']=nomes_produtos
        df1['Preço com desconto']=p_des
        df1['Preço sem desconto']=p_s_des
        df1['Desconto']=DESCONTO

        df=pd.concat([df,df1])


372 HYPERA
287 ACHE RX
238 EUROFARMA RX
228 EMS GENERICO
187 CIMED
180 SIGMA PHARMA RX
152 GERMED GENERICO
152 BIOLAB RX
149 LIBBS
135 APSEN
121 MEDLEY GENERICOS
120 SUPERA
109 MOMENTA
109 FARMOQUIMICA
109 EUROFARMA GENERICO
100 MYRALIS
83 ABBOTT RX
75 TEUTO GENERICO
74 SANOFI RX
74 ACHE OTC
69 TORRENT
68 CRISTALIA/LATINOFARMA
66 SANOFI OTC
65 PRATI DONADUZZI GENERICO
63 UNIAO QUIMICA GENOM
61 UNIAO QUIMICA SIMILAR
60 NEO QUIMICA GENERICO
58 BIOLAB GENERICO
55 PHARLAB
55 NEO QUIMICA SIMILAR
54 LEGRAND GENERICO
52 MARJAN
52 GSK OTC
51 ZANPHY
50 NOVA QUIMICA GENERICO
49 NOVARTIS RX
49 GEOLAB GENERICO
45 GSK RX
43 PROCTER  GAMBL
41 UNIAO QUIMICA GENERICO
39 MERCK RX
38 BIOSINTETICA GENERICO
37 ALLERGAN
36 ALCON
34 TEUTO SIMILAR
34 SERVIER
33 BAYER RX
32 HERBARIUM
32 GLOBO GENERICO
32 ASTRAZENECA
31 SANDOZ GENERICO
31 CELLERA
31 BIOLAB AVERT OTC
29 DIVCOM
28 ZODIAC ADIUM
28 SANKYO PHARMA
28 RANBAXY
28 MEGALABS
28 EMS SIMILAR
28 BAYER OTC
27 BOEHRINGER
27 AIRELA
26 TAKEDA
25 PFIZER
24 ZAMBO

In [8]:

df.reset_index(inplace=True,drop=True)
df


,EAN,Marca,Nome,Preço com desconto,Preço sem desconto,Desconto
0,7896094920453,hypera,Macrodantina 100mg com 40 Cápsulas,22.50,23.94,1.44
1,7891142142290,hypera,Diprogenta Creme 30g,43.72,50.84,7.12
2,7891142142283,hypera,Diprogenta Pomada 30g,45.43,52.83,7.40
3,7891142165770,hypera,Macrodantina 100mg com 28 Cápsulas,15.79,16.80,1.01
4,7891142180766,hypera,Quadriderm Creme 20g,56.28,59.87,3.59
...,...,...,...,...,...,...
6105,7898442102702,aura,Acetilcisteina xpe ad sabor cereja 120ml,13.09,20.99,7.90
6106,7898037080354,almeida-prado-ltda,Complexo Homeopático Nº 35 com 60 Comprimidos,18.29,19.99,1.70
6107,7898911244643,abl-brasil,Cefalexina 500mg com 8 Comprimidos,11.56,44.02,32.46
6108,7898716456135,a2f,Hidrafito Caixa com 4 Unidades,13.49,13.49,0.00


In [9]:

df.to_excel("MINAS_MAIS.xlsx")


In [6]:
#teoricamente ta funcionando (fica de backup)


""" colunas=['EAN','Marca','Nome','Preço com desconto','Preço sem desconto','Desconto']
df=pd.DataFrame(columns=colunas)


url='https://www.drogariasminasmais.com.br/medicamentos'
html=pegar_html(url)
marcas=achar_marcas(html)
LINKS=link(marcas)
MARCAS=achar_marcas_sem_link(html)


for j in range(len(LINKS)):
    json_pag=get_json(LINKS[j] + "1")
    json_generico=get_json(LINKS[0] + "1")
    pag=qtd_produtos(json_generico,MARCAS[j])
    print(pag,MARCAS[j])
    pag=pag/15
    pag=math.ceil(pag) + 1
    
    for i in range(1,pag):

        colunas=['EAN','Marca','Nome','Preço com desconto','Preço sem desconto','Desconto']
        df1=pd.DataFrame(columns=colunas)


        json_pag=get_json(LINKS[j] + str(i))
        ean=get_ean(json_pag)
        nomes_produtos=get_name(json_pag)
        p_des=get_preço_c_desconto(json_pag)
        p_s_des=get_preço_s_desconto(json_pag)
        DESCONTO=desconto(p_s_des,p_des)

        df1['EAN']=ean
        df1['Marca']=marcas[j]
        df1['Nome']=nomes_produtos
        df1['Preço com desconto']=p_des
        df1['Preço sem desconto']=p_s_des
        df1['Desconto']=DESCONTO

        df=pd.concat([df,df1]) """

' colunas=[\'EAN\',\'Marca\',\'Nome\',\'Preço com desconto\',\'Preço sem desconto\',\'Desconto\']\ndf=pd.DataFrame(columns=colunas)\n\n\nurl=\'https://www.drogariasminasmais.com.br/medicamentos\'\nhtml=pegar_html(url)\nmarcas=achar_marcas(html)\nLINKS=link(marcas)\nMARCAS=achar_marcas_sem_link(html)\n\n\nfor j in range(len(LINKS)):\n    json_pag=get_json(LINKS[j] + "1")\n    json_generico=get_json(LINKS[0] + "1")\n    pag=qtd_produtos(json_generico,MARCAS[j])\n    print(pag,MARCAS[j])\n    pag=pag/15\n    pag=math.ceil(pag) + 1\n    \n    for i in range(1,pag):\n\n        colunas=[\'EAN\',\'Marca\',\'Nome\',\'Preço com desconto\',\'Preço sem desconto\',\'Desconto\']\n        df1=pd.DataFrame(columns=colunas)\n\n\n        json_pag=get_json(LINKS[j] + str(i))\n        ean=get_ean(json_pag)\n        nomes_produtos=get_name(json_pag)\n        p_des=get_preço_c_desconto(json_pag)\n        p_s_des=get_preço_s_desconto(json_pag)\n        DESCONTO=desconto(p_s_des,p_des)\n\n        df1[\'EAN\'